# AI / ML Easy Button 
## *Basics using Review Data*
Welcome to the AI/ML Easy Button. This notebook will allow you to explore to explore Snowflake's out of the box functions and understand how they work with custom prompts in both SQL and Python.  
We will leverage some Take-Two Game Review Data and Game Summary Data for these analysis.

The Snowflake Docs are always a great reference point, so we will include these along the way.
[LLM Docs](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

![](https://www.fatherhood.org/hs-fs/hubfs/Images/Blog/easy-button.png?width=585&name=easy-button.png)

In [ ]:
# !!! Add packages in the upper right under packages --- this code needs 'snowflake-ml'
# Import python packages
import streamlit as st
import pandas as pd
import json

# We can also use Snowpark for our analyses!
from snowflake.snowpark import functions as F
from snowflake.cortex import Complete, Sentiment, Summarize, Translate
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
--first create your own schema to work in!
create schema if not exists MY_NAME ;
use schema MY_NAME;
---this should be your name!!
select current_database(), current_schema();

### Translate  
https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex

In [ ]:
-- TRANSLATE
-- Translate text from Reviews

---call the in built SQL function, with text, from language and to language
select
    snowflake.cortex.translate(
        'Ser-lhe-ei honesto, tive dificuldade em comprar o GTA V pela terceira vez (PS3, PS4 e agora PS5) mas pelo preço é realmente uma atualização digna da próxima geração. O jogo no PlayStation 5 é verdadeiramente notável',
        'pt',
        'en');

In [ ]:
-- SQL Translate from Any Text to English...leave from language blank to infer it
select
    snowflake.cortex.translate(
        '마이크로 트랜잭션이 매우 공격적임에도 불구하고 모든 스포츠 게임 팬에게 NBA 2K23를 안전하게 추천할 수 있습니다. 게임 플레이가 너무 좋고 향후 업데이트에서 문제가 해결되지 않기를 바랍니다.',
        '',
        'en');

In [ ]:
#PYTHON IS EASY TOO...Now we use the same function directly from Python
Translate(
    "Detta är det bästa videospelet någonsin!",
    "",
    "en"
)

In [ ]:
--we can run this against millions of records easily, effeciently and securly
select
    REVIEW_TEXT,
    snowflake.cortex.translate(
        REVIEW_TEXT,
        '',
        'en'
    ) as review_in_english
from
    LAB_DATA.PUBLIC.REVIEWS
    where IS_ENGLISH = 0;

In [ ]:
###can turn cell results into snowpark dataframes... or Pandas ones too
sp_df = sample_reviews.to_df()
sp_df.show(3)
pd_df = sample_reviews.to_pandas()
pd_df.head(3)

### Sentiment 
https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex

In [ ]:
select
    REVIEW_TEXT,
    snowflake.cortex.sentiment(
        snowflake.cortex.translate( ---chained together, translate and sentiment
        REVIEW_TEXT,
        '',
        'en'
        ) 
    ) as review_sentiment
from
    LAB_DATA.PUBLIC.REVIEWS

### Summarize  
https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex

In [ ]:
select
    snowflake.cortex.summarize('For me, Grand Theft Auto V’s extraordinary scope is summed up in two favourite moments. One is from a mid-game mission in which I flew a plane into another plane, fought the crew, hijacked the thing, and then parachuted out and watched it crash into the sea to escape death at the hands of incoming military fighter jets. Another time, whilst driving around in an off-road buggy, I got distracted by something that looked like a path up one of the San Andreas mountains. Turns out it was a path, and I spent 15 minutes following to the summit, where I nearly ran over a group of hikers. “Typical!” one of them yelled at me, as if he nearly gets run over by a rogue ATV on top of a mountain every time he goes on a hike.

I could go on like this for ages. GTA V has an abundance of such moments, big and small, that make San Andreas – the city of Los Santos and its surrounding areas – feel like a living world where anything can happen. It both gives you tremendous freedom to explore an astonishingly well-realised world and tells a story that’s gripping, thrilling, and darkly comic. It is a leap forward in narrative sophistication for the series, and there’s no mechanical element of the gameplay that hasn’t been improved over Grand Theft Auto IV. It’s immediately noticeable that the cover system is more reliable and the auto-aim less touchy. The cars handle less like their tires are made of butter and stick better to the road, though their exaggerated handling still leaves plenty of room for spectacular wipeouts. And at long last, Rockstar has finally slain one of its most persistent demons, mission checkpointing, ensuring that you never have to do a long, tedious drive six times when you repeatedly fail a mission ever again.


Grand Theft Auto V is also an intelligent, wickedly comic, and bitingly relevant commentary on contemporary, post-economic crisis America. Everything about it drips satire: it rips into the Millennial generation, celebrities, the far right, the far left, the middle class, the media... Nothing is safe from Rockstar’s sharp tongue, including modern video games. One prominent supporting character spends most of his time in his room shouting sexual threats at people on a headset whilst playing a first-person shooter called Righteous Slaughter (“Rated PG – pretty much the same as the last game.”) It’s not exactly subtle – he literally has the word “Entitled” tattooed on his neck, and the in-game radio and TV’s outright piss-takes don’t leave much to the imagination – but it is often extremely funny, and sometimes provocative with it. Grand Theft Auto’s San Andreas is a fantasy, but the things it satirises – greed, corruption, hypocrisy, the abuse of power – are all very real. If GTA IV was a targeted assassination of the American dream, GTA V takes aim at the modern American reality. The attention to detail that goes into making its world feel alive and believable is also what makes its satire so biting.

Grand Theft Auto V’s plot happily operates at the boundaries of plausibility, sending you out to ride dirt bikes along the top of trains, hijack military aircraft, and engage in absurd shootouts with scores of policemen, but its three main characters are what keep it relatable even at its most extreme. The well-written and acted interplay between them provides the biggest laughs and most affecting moments, and the way that their relationships with one another developed and my opinion of them changed throughout the story gave the narrative its power. They feel like people – albeit extraordinarily f***ed-up people.

Michael is a retired con man in his 40s, filling out around the middle as he drinks beside the pool in his Vinewood mansion with a layabout son, air-headed daughter, serially unfaithful wife, and very expensive therapist – all of whom hate him. Franklin is a young man from downtown Los Santos who laments the gang-banger stereotype even as he’s reluctantly seduced by the prospect of a bigger score. And then there’s Trevor, a volatile career criminal who lives in the desert selling drugs and murdering rednecks; a psychopath whose bloodthirsty lunacy is fuelled by a combination of methamphetamine and a seriously messed-up childhood.


The missions flit between their individual stories and an overarching plotline that involves all three, and it’s a credit to GTA V’s versatility and universal quality that each character has his share of standout missions. As their arcs developed I felt very differently about each of them at different times – they’re not entirely the archetypes that they seem to be.

This three-character structure makes for excellent pacing and great variety in the storyline, but it also allows Rockstar to compartmentalise different aspects of Grand Theft Auto’s personality. In doing so, it sidesteps some of the troubling disconnect that arose when Niko Bellic abruptly alternated between anti-violent philosophising and sociopathic killing sprees in GTA IV. Here, many of Michael’s missions revolve around his family and his past, Franklin is usually on call for vehicular mayhem, and extreme murderous rampages are left to Trevor. Each has a special ability suited to his skills – Franklin can to slow time while driving, for example – which gives them a unique touch. Narratively, it’s effective – even off-mission I found myself playing in character, acting like a mid-life-crisis guy with anger issues as Michael, a thrill-seeker as Franklin, and a maniac as Trevor. The first thing I did when Franklin finally made some good money was buy him an awesome car, because I felt like that’s what he’d want.

Trevor feels a like a bit of a get-out-of-jail-free card for Rockstar, providing an outlet for all the preposterous antics and murderous behaviour that otherwise might not fit in with GTA V’s narrative ambitions. I found his violent insanity a little overblown and tiresome at first. As get-out clauses go, though, it’s pretty effective, and Trevor’s over-the-top missions are some of GTA V’s action-packed highlights. It’s a successful way of solving a problem that’s prevalent in open-world games: the tension between the story that the writers are trying to tell, and the story you create yourself within its systems and its world. Grand Theft Auto V accommodates both, masterfully, allowing neither to undermine the other.


The actual act of switching between them also provides a window into their individual lives and habits, fleshing out their personalities in a way that feels natural and novel. Pick a character and the camera zooms out over the San Andreas map, closing back in on wherever they happen to be. Michael might be at home watching TV when you drop in on him, or speeding along the motorway blasting ‘80s hits, or having a cigarette at the golf club; Franklin might be walking out of a strip club, munching a bag of snacks at home, or arguing with his ex-girlfriend; there’s a good chance that Trevor could be passed out half naked on a beach surrounded by dead bodies or, on one memorable occasion, drunk in a stolen police helicopter.

It could be nearly anything, because there is a bewildering multiplicity of things to do in the new San Andreas – tennis, yoga, hiking, racing on sea and on land, flying planes, golfing, cycling, diving, hunting, and more. The missions are an able guide to both San Andreas’ locations and its activities, touring you around the map and whetting your appetite for independent exploration of it all. The way that we’re introduced to San Andreas never feels artificial – the map is completely open from the start, for example – which contributes to the impression that it’s a real place, somewhere you can get to know. If GTA IV’s Liberty City feels like a living city, San Andreas feels like a living world. I saw people walking their dogs along the beach in the country as I jet-skied past, arguing on the street outside a cinema in Los Santos, and camped – with tents and everything – overnight on Mount Chiliad, before packing up and continuing a hike in the morning. It’s astounding.

The ambience changes dramatically depending on where you are, too. Trevor’s dusty trailer out in the middle of nowhere in Blaine County feels like a different world from downtown Los Santos or Vespucci Beach. It wasn’t until the first time I flew a plane out of the city and over the mountains I was cycling around a few hours before that the full scale of it became obvious. It pushes the Xbox 360 and PlayStation 3 further than it has any right to, and it looks incredible. The biggest jump in quality since Grand Theft Auto IV is the character animation, but the world is also much more expansive, detailed, and populous. The price we pay for that is occasional framerate dips and texture pop-in, which I found became more prominent the longer I played, but never significantly detracted from my experience. For such a gigantic and flexible world it’s also remarkably bug-free – I encountered just three minor issues in the 35 hours I spent on my first playthrough, none of which caused me to fail a mission.


San Andreas’s extraordinary sense of place is heightened by the fact that so much of it isn’t on the map. There’s so much going on that it’s easy to find things organically, rather than spend your life following a mission marker. I once stole a passenger jet from the airport for the hell of it, then parachuted onto the top of the tallest building in Los Santos. (I then accidentally jumped off the top and fell to my death, forgetting that I’d already used the parachute, but I usually leave that bit out.) Out driving in the country, I came across a man tied to a telephone pole in womens’ underwear. I chased down criminals who randomly swipe purses on the street, and happened across gunbattles between police and other miscreants, events that add a sense that this world isn’t completely uneventful if I wasn’t here to disrupt normalcy. I bought an expensive mountain bike and cycled around in the hills, enjoying the view. These little moments can be captured on your phone camera – which, brilliantly, can also take selfies. I have several snaps of Trevor doing his unhinged version of a smile in his underpants on top of a mountain.

The story that GTA V tells through its missions takes full advantage of all this variety beyond driving and shooting (though the driving and shooting is still supremely enjoyable). It’s got so many great moments. It had me racing Michael’s lazy blob of a son across Vespucci Beach in one of many misguided attempts at father-son bonding, using a thermal scope to search for someone from a helicopter before chasing them across the city on the ground, torching a meth lab, towing cars for Franklin’s crack-addict cousin to prevent him from losing his job, infiltrating a facility from the sea in a wetsuit and flippers, piloting a submarine, impersonating a construction worker, doing yoga, escaping on jet skis, failing multiple times to land a plane loaded with drugs at a hangar out in the desert… it goes on and on. The days of a repetitive series of “drive here, find this guy, shoot this guy” are behind us. Even missions that would otherwise be formulaic are imbued with novelty and excitement by the potential to play them from three different viewpoints – in a shootout, Trevor might be firing RPGs from a rooftop as Michael and Franklin flank the enemy on the ground.

It’s the heists – multi-stage, huge-scale events that serve as the story’s climactic peaks – that show Grand Theft Auto V at its most ambitious and accomplished. Usually there’s a choice between a more involved, stealthier option that will (hopefully) attract less heat, and an all-out option that will be less tense but more explosively chaotic – and what crew to take along with you on the job. All of GTA V’s missions are replayable at any time, letting you relive favourite moments or try out another approach. They also have optional objectives in the vein of Assassin’s Creed’s synchronisation challenges, but crucially, these are invisible the first time you play a mission, and so they don’t distract you from doing things your own way.


Sometimes your own way won’t be the way that the designers expected you to do something, and though Grand Theft Auto V is usually very good at bending around you when that happens, there were one or two occasions where it wasn’t prepared for my personal brand of chaos. Overtake a car you’re not supposed to overtake and it will zip through lines of traffic as if by magic. Despite the introduction of new stealth mechanics, enemies will miraculously see you when the mission dictates that they should. Kill someone before you’re supposed to, and that’s sometimes Mission Failed. Most of the time the scripting is good enough to be invisible, but when it’s not, you really notice it – if only because most of the time it’s so seamless.

As ever, some of the wittiest writing shows up on the in-game radio that plays behind all of the exploration and mayhem. “There’s nothing more successful, more masculine, more American than a big wad of cash,” blasts one of the in-game ads. “We know times are tough, but they don’t have to be tough for you. Still got some liquidity in your house? Are you insane?” The music selection is also typically excellent, leading to many of those serendipitous moments where you’re driving along and the perfect song comes on. During a heist, when the radio isn’t blaring the background, a dynamic soundtrack seriously builds tension.

The satire is helped by integration of modern life into the game world. Every character revolves around their smartphone – it’s used to trade stocks, call up friends to meet up and send emails. There’s a great Facebook spoof, Life Invader, on the in-game Interne, with the slogan “Where Your Personal Information Becomes A Marketing Profile (That We Can Sell)”. You’ll hear adverts for preposterous parodic TV shows that you can actually watch on your TV at home, optionally whilst enjoying a toke. It might not be realistic, but it definitely feels authentic.


It’s worth mentioning that when it comes to sex, drugs, and violence, GTA V pushes boundaries much further than ever before. If the morality police were worried about Hot Coffee, there’s a lot here that will provoke moral hysteria. It’s deliciously subversive, and firmly tongue in cheek... but once or twice, it pushes the boundaries of taste, too. There’s one particular scene, a torture scene in which you have no choice but to actively participate, that I found so troubling that I had difficulty playing it; even couched in obvious criticism of the US government’s recourse to torture post 9/11, it’s a shocking moment that will attract justified controversy. It brings to mind Call of Duty: Modern Warfare 2’s No Russian mission, except worse, and without the option to skip over it. Some other stuff, like the ever-present prostitution and extensive strip-club minigames, feels like it’s there just because it can be rather than because it has anything to say.

There is nothing in San Andreas, though, that doesn’t serve Rockstar’s purpose in creating an exaggerated projection of America that’s suffused with crime, violence and sleaze. There are no good guys in GTA V. Everyone you meet is a sociopath, narcissist, criminal, lunatic, sadist, cheat, liar, layabout, or some combination of those. Even a man who pays good money to assassinate Los Santos’ worst examples of corporate greed is playing the stock market to his advantage whilst he does it. In a world like this, it’s not hard to see why violence is so often the first recourse. All the pieces fit.


');

### COMPLETE  
https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex

In [ ]:
---Need to leverage an LLM... this is as easy as SELECT too. Models are hosted and managed by Snowflake in a secure & scalable environment, just call the built-in function.  

select
   REVIEW_TEXT, snowflake.cortex.complete(
        'llama3-8b',
        '[INST]
### 
Tell me based on the following game reviews, would the reviewer recommend the game to \
their friends and family? Answer should be only one of the following words - \
"Likely" or "Unlikely" or "Unsure". Make sure there are no additional additional text.
Review -
###' || REVIEW_TEXT) as CLEANED_REVIEWS
     from  LAB_DATA.PUBLIC.REVIEWS limit 5

In [ ]:
---Need to leverage an LLM... this is as easy as SELECT too. Models are hosted and managed by Snowflake in a secure & scalable environment, just call the built-in function.  

select
   REVIEW_TEXT, snowflake.cortex.complete(
        'llama3-8b',
        '[INST]
### 
Tell me based on the following game reviews, is there any toxic or offensive language in the text? Answer should be only one of the following words - \
"Likely Toxic" or "Unlikely Toxic" or "Unsure". Make sure there are no additional additional text.
Review -
###' || REVIEW_TEXT) as CLEANED_REVIEWS
     from  LAB_DATA.PUBLIC.REVIEWS limit 5

In [ ]:
 
select
    snowflake.cortex.complete(
        'mistral-large',
        'What are RockStar\'s three most popular video games?  Return the answer in consice a JSON array object with NAME and RANK'
    );

In [ ]:
#Similarly we can call this easily from Python
topTitles = Complete(
        "mistral-large",
        "What are Rockstar's three most popular video games?  Display answer in consice a JSON array object with NAME and RANK"
    )
#can manipulate this json object
topTitlesJSON = json.loads(topTitles)
for title in topTitlesJSON:
    print(str(title["RANK"])+' '+title["NAME"])

In [ ]:
select * from LAB_DATA.PUBLIC.GAMES limit 10

In [ ]:
---SQL to run the LLM over a table
select
    title,
    summary,
    snowflake.cortex.complete(
        'llama3-8b',
        CONCAT(
            'You\'re a game critic. I really like role playing games but I don\'t like violience. Based on the summary between <summary> and </summary>, can you tell me if 
            this would be a good game for me? <summary>', summary, '</summary>'
        )
    ) as answer
from
    LAB_DATA.PUBLIC.VW_GAMES
where developer = 'Rockstar Games'
limit 5;

In [ ]:
#PYTHON to run the LLM over a table. 
df = session.table("LAB_DATA.PUBLIC.VW_GAMES").filter(F.col('DEVELOPER') == 'Rockstar Games').select(
    F.col("title"),
    Complete(
        "llama3-8b",
        F.concat(
            F.lit("""Based on the value between the <context> and </context> tags, please provide me with a list
            of 3 similar games. Return the result as only a JSON array.  Be conscise.
            <context>"""),
            F.col("summary"),
            F.lit("</context>"))
    ).alias("similar_titles")
).limit(5)
df.show()

In [ ]:
###easily save to table in SQL or Python
df.write.mode("overwrite").save_as_table("GAMES_SIMILAR")

In [ ]:
select * from GAMES_SIMILAR limit 5;

### EMBED_TEXT
https://docs.snowflake.com/sql-reference/functions/embed_text-snowflake-cortex

In [ ]:
select
    snowflake.cortex.embed_text_768(
        'snowflake-arctic-embed-m',
        'I love video games'
    );

In [ ]:
select
    summary,
    snowflake.cortex.embed_text_768(
        'snowflake-arctic-embed-m',
        summary
    ) as summary_embedding
from
    LAB_DATA.PUBLIC.GAMES
limit 10;

### VECTOR DISTANCE CALCULATIONS  
https://docs.snowflake.com/en/sql-reference/functions/vector_cosine_similarity

In [ ]:
select
    vector_cosine_similarity(
        snowflake.cortex.embed_text_768('snowflake-arctic-embed-m', 'California Contemporary style'),
        snowflake.cortex.embed_text_768('snowflake-arctic-embed-m', 'California Contemporary style homes')
    );

In [ ]:
--snowflake own SOTA text embed model
create or replace table GAMES_EMBED as select *,
    snowflake.cortex.embed_text_768(
        'snowflake-arctic-embed-m',
        summary
    ) as summary_embedding
from
    LAB_DATA.PUBLIC.GAMES

In [ ]:
--use cosine similarity to find like titles
select
    b.title SIMILAR_TITLES, a.title TITLE_SEARCHED, 
    vector_cosine_similarity(
        a.summary_embedding,
        b.summary_embedding
    ) as similarity
from
    GAMES_EMBED a
    cross join GAMES_EMBED b
where
    a.title = 'NBA 2K23'
order by
    similarity desc
limit 20;

## Retrieval-Augmented Generation (RAG) 
is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response.
We do this in two steps when querying the LLM
1. First we take the question and get the relevant data using vector cosine similarities (as shown above)
2. Next we use that relevant data in the prompt to the LLM with our question

In [ ]:
#question = """What Cities does GTA V Take Place In?"""
#question = """Is Lebron James featured in any NBA 2K games?"""
question = """What year is Grand Theft Auto: Vice City set in?"""

In [ ]:
#step 1 get the relvant data
relevant_titles = session.sql(f"""
   select
            title, summary,
            vector_cosine_similarity(
                summary_embedding,
                snowflake.cortex.embed_text_768(
                  'snowflake-arctic-embed-m',
                  '{question}'
                )
            ) as similarity
        from
            GAMES_EMBED
  order by
      similarity desc
  limit 10""") 
relevant_titles.show()

In [ ]:
#step 2 feed to the LLM using a specific prompt
info = '. | '.join([x[0] for x in relevant_titles.select("*").collect()]).replace("'", "")
prompt = f"""
            You are a video game expert. Please provide knowledge and guidance to the questions in the tags <question> and </question> based on the provided 
            context found between the tags <context> and </context>.

            <context>
            '{info}'
            </context>
            <question>
            '{question}'
            </question>
            Answer: """
query = """
      select
          snowflake.cortex.complete(
              ?, 
              ?
          ) as response
      """
complete = session.sql(query, params=['mistral-large', prompt])
with st.chat_message(name="Assistant"):
    st.write(complete.collect()[0][0])

# UI to Easily Create Chat Bots with (RAG & Search) is coming!
A Summit our head of product randomly chose someone from the audience to create a chatbot in Snowflake.  They had only logged into Snowflake 7 times and were able to create a RAG chat bot using the new UI in minutes.

## Fine Tuning Too
Snowflake is making this easy too, you supply your training data to the model via a table and you get an additional version of the model...fine tuned to your task.
Fine tuning can be the best combo of results and cost. Dont have training data?  Use a more expensive LLM to score a set of data and train a smaller, cheaper one. Boom cheap and effective.

[Snowflake Fine Tuning Docs](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-finetuning?_fsi=wWDeqSCS&_fsi=wWDeqSCS)

[Snowflake Fine Tuning Workshop](https://quickstarts.snowflake.com/guide/finetuning_llm_using_snowflake_cortex_ai/index.html?index=..%2F..index#2)